<a href="https://colab.research.google.com/github/Pragati-cloud/Pragati-cloud/blob/main/templates/aistudio_gemini_prompt_freeform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Install & import

In [ ]:
!pip install -U -q google-generativeai

In [1]:
# Install the client library and import necessary modules.
import google.generativeai as genai

import base64
import copy
import hashlib
import io
import json
import mimetypes
import pathlib
import pprint
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

## Set the API key

Add your API_KEY to the secrets manager in the left panel "🔑".

In [2]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

SecretNotFoundError: Secret GOOGLE_API_KEY does not exist.

In [ ]:
# Configure the client library by providing your API key.
genai.configure(api_key=GOOGLE_API_KEY)

## Parse the arguments

In [3]:
model = 'gemini-1.5-flash' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6IkNyZWF0ZSBhIHdlYiBhcHAgY2FsbGVkIFBvcnRmb2xpb1xcbjEubmF2aWdhdGlvbiBiYXIgY29udGFpbmluZyBwcm9qZWN0ICwgY29udGFjdCBtZSAsIHNlcnZpY2VzLlxcbjIuIEl0IHNob3VsZCBiZSBibGFjayBpbiBibHVlIGluIGNvbG9yICwgYmFja2dyb3VuZCBzaG91bGQgY29udGFpbiBhbiBhc3Ryb25vdXQgcGljdHVyZSAuXFxuXFxuNC4gSXQgc2hvdWxkIGJlIGEgc2luZ2xlIEhUTUwgZmlsZSwgd2l0aCBubyBzZXBhcmF0ZSBKUyBvciBDU1MgZmlsZXMuXFxuNS4gSXQgc2hvdWxkIHNheSByaWdodHMgcmVzZXJ2ZWQgaW4gdGhlIGZvb3Rlci5cXG42LiBEbyBub3QgdXNlIGFueSB1bmljb2RlIGNoYXJhY3RlcnMuIn1dfSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjpbeyJ0ZXh0IjoiYGBgaHRtbFxcbjwhRE9DVFlQRSBodG1sPlxcbjxodG1sPlxcbjxoZWFkPlxcbjx0aXRsZT5Qb3J0Zm9saW88L3RpdGxlPlxcbjxzdHlsZT5cXG5ib2R5IHtcXG4gIGJhY2tncm91bmQtaW1hZ2U6IHVybChcXFwiaHR0cHM6Ly9pbWFnZXMudW5zcGxhc2guY29tL3Bob3RvLTE2MzY1NzEwOTEyMTItMWMwNjQ3ZTA0MTE1P2l4bGliPXJiLTQuMC4zJml4aWQ9TTN3eE1qQTNmREI4TUh4elpXRnlZMmg4TW54OFlYTjBjbTl1WVhVZ2ZIeGxibnd3Zkh3d2ZIdyUzRCZhdXRvPWZvcm1hdCZmaXQ9Y3JvcCZ3PTUwMCZxPTYwXFxcIik7IC8qIFJlcGxhY2Ugd2l0aCB5b3VyIGFzdHJvbmF1dCBpbWFnZSBVUkwgKi9cXG4gIGJhY2tncm91bmQtc2l6ZTogY292ZXI7XFxuICBiYWNrZ3JvdW5kLXJlcGVhdDogbm8tcmVwZWF0O1xcbiAgY29sb3I6IHdoaXRlO1xcbiAgZm9udC1mYW1pbHk6IHNhbnMtc2VyaWY7XFxuICBtYXJnaW46IDA7XFxufVxcblxcbi5jb250YWluZXIge1xcbiAgbWF4LXdpZHRoOiA4MDBweDtcXG4gIG1hcmdpbjogMCBhdXRvO1xcbiAgcGFkZGluZzogMjBweDtcXG59XFxuXFxuaGVhZGVyIHtcXG4gIGJhY2tncm91bmQtY29sb3I6ICMwMDI2NGQ7XFxuICBwYWRkaW5nOiAyMHB4IDA7XFxuICB0ZXh0LWFsaWduOiBjZW50ZXI7XFxufVxcblxcbm5hdiB1bCB7XFxuICBsaXN0LXN0eWxlOiBub25lO1xcbiAgbWFyZ2luOiAwO1xcbiAgcGFkZGluZzogMDtcXG59XFxuXFxubmF2IGxpIHtcXG4gIGRpc3BsYXk6IGlubGluZS1ibG9jaztcXG4gIG1hcmdpbjogMCAxMHB4O1xcbn1cXG5cXG5uYXYgYSB7XFxuICBjb2xvcjogd2hpdGU7XFxuICB0ZXh0LWRlY29yYXRpb246IG5vbmU7XFxuICBwYWRkaW5nOiA1cHggMTBweDtcXG4gIGJvcmRlci1yYWRpdXM6IDVweDtcXG4gIHRyYW5zaXRpb246IGJhY2tncm91bmQtY29sb3IgMC4zcztcXG59XFxuXFxubmF2IGE6aG92ZXIge1xcbiAgYmFja2dyb3VuZC1jb2xvcjogIzAwNDA4MDtcXG59XFxuXFxuZm9vdGVyIHtcXG4gIGJhY2tncm91bmQtY29sb3I6ICMwMDI2NGQ7XFxuICBwYWRkaW5nOiAxMHB4IDA7XFxuICB0ZXh0LWFsaWduOiBjZW50ZXI7XFxuICBwb3NpdGlvbjogZml4ZWQ7XFxuICBib3R0b206IDA7XFxuICB3aWR0aDogMTAwJTtcXG59XFxuXFxuPC9zdHlsZT5cXG48L2hlYWQ+XFxuPGJvZHk+XFxuXFxuPGhlYWRlcj5cXG4gIDxoMT5NeSBQb3J0Zm9saW88L2gxPlxcbjwvaGVhZGVyPlxcblxcbjxkaXYgY2xhc3M9XFxcImNvbnRhaW5lclxcXCI+XFxuXFxuICA8bmF2PlxcbiAgICA8dWw+XFxuICAgICAgPGxpPjxhIGhyZWY9XFxcIiNcXFwiPlByb2plY3RzPC9hPjwvbGk+XFxuICAgICAgPGxpPjxhIGhyZWY9XFxcIiNcXFwiPkNvbnRhY3QgTWU8L2E+PC9saT5cXG4gICAgICA8bGk+PGEgaHJlZj1cXFwiI1xcXCI+U2VydmljZXM8L2E+PC9saT5cXG4gICAgPC91bD5cXG4gIDwvbmF2PlxcblxcbiAgPG1haW4+XFxuICAgIDwhLS0gWW91ciBwb3J0Zm9saW8gY29udGVudCBoZXJlIC0tPlxcbiAgPC9tYWluPlxcblxcbjwvZGl2Plxcblxcbjxmb290ZXI+XFxuICAmY29weTsgMjAyMyBBbGwgUmlnaHRzIFJlc2VydmVkLlxcbjwvZm9vdGVyPlxcblxcbjwvYm9keT5cXG48L2h0bWw+XFxuYGBgXFxuXFxuKipFeHBsYW5hdGlvbjoqKlxcblxcbjEuICoqSFRNTCBTdHJ1Y3R1cmU6KipcXG4gICAtIEJhc2ljIEhUTUwgc3RydWN0dXJlIHdpdGggYGhlYWRgIGFuZCBgYm9keWAgZWxlbWVudHMuXFxuICAgLSBgaGVhZGVyYCBmb3IgdGhlIHRpdGxlIGFuZCBuYXZpZ2F0aW9uLlxcbiAgIC0gYG5hdmAgZm9yIHRoZSBuYXZpZ2F0aW9uIGJhci5cXG4gICAtIGBtYWluYCBmb3IgeW91ciBwb3J0Zm9saW8gY29udGVudCAoYWRkIHlvdXIgYWN0dWFsIGNvbnRlbnQgaGVyZSkuXFxuICAgLSBgZm9vdGVyYCBmb3IgY29weXJpZ2h0IGluZm9ybWF0aW9uLlxcblxcbjIuICoqQ1NTIFN0eWxpbmc6KipcXG4gICAtICoqYGJvZHlgOioqXFxuICAgICAtIFNldHMgYmFja2dyb3VuZCBpbWFnZSB1c2luZyBgdXJsKClgLiBSZXBsYWNlIHRoZSBwbGFjZWhvbGRlciBVUkwgd2l0aCB5b3VyIGFjdHVhbCBhc3Ryb25hdXQgaW1hZ2UgVVJMLlxcbiAgICAgLSBTZXRzIGJhY2tncm91bmQgc2l6ZSBhbmQgcmVwZWF0LlxcbiAgICAgLSBTZXRzIHRleHQgY29sb3IgdG8gd2hpdGUuXFxuICAgICAtIFNldHMgZm9udCBmYW1pbHkgdG8gc2Fucy1zZXJpZi5cXG4gICAgIC0gUmVtb3ZlcyBkZWZhdWx0IG1hcmdpbnMuXFxuICAgLSAqKmAuY29udGFpbmVyYDoqKlxcbiAgICAgLSBTZXRzIG1heGltdW0gd2lkdGggZm9yIGNvbnRlbnQuXFxuICAgICAtIENlbnRlcnMgdGhlIGNvbnRlbnQgaG9yaXpvbnRhbGx5LlxcbiAgICAgLSBBZGRzIHBhZGRpbmcgZm9yIHNwYWNpbmcuXFxuICAgLSAqKmBoZWFkZXJgOioqXFxuICAgICAtIFNldHMgYSBkYXJrIGJsdWUgYmFja2dyb3VuZCBjb2xvci5cXG4gICAgIC0gQWRkcyBwYWRkaW5nIGFuZCBjZW50ZXJzIHRoZSB0ZXh0LlxcbiAgIC0gKipgbmF2YDoqKlxcbiAgICAgLSBSZW1vdmVzIGxpc3QgYnVsbGV0cy5cXG4gICAgIC0gU2V0cyBkaXNwbGF5IHRvIGlubGluZS1ibG9jayBmb3IgaG9yaXpvbnRhbCBuYXZpZ2F0aW9uLlxcbiAgIC0gKipgbmF2IGxpYDoqKlxcbiAgICAgLSBBZGRzIG1hcmdpbiBiZXR3ZWVuIG5hdmlnYXRpb24gaXRlbXMuXFxuICAgLSAqKmBuYXYgYWA6KipcXG4gICAgIC0gU2V0cyB0ZXh0IGNvbG9yIHRvIHdoaXRlLlxcbiAgICAgLSBSZW1vdmVzIHVuZGVybGluZXMuXFxuICAgICAtIEFkZHMgcGFkZGluZyBhbmQgcm91bmRlZCBjb3JuZXJzLlxcbiAgICAgLSBBZGRzIGEgaG92ZXIgZWZmZWN0IHRvIGNoYW5nZSBiYWNrZ3JvdW5kIGNvbG9yLlxcbiAgIC0gKipgZm9vdGVyYDoqKlxcbiAgICAgLSBTZXRzIGEgZGFyayBibHVlIGJhY2tncm91bmQgY29sb3IuXFxuICAgICAtIEFkZHMgcGFkZGluZyBhbmQgY2VudGVycyB0aGUgdGV4dC5cXG4gICAgIC0gRml4ZXMgaXQgdG8gdGhlIGJvdHRvbSBvZiB0aGUgc2NyZWVuLlxcbiAgICAgLSBNYWtlcyBpdCBmdWxsIHdpZHRoLlxcblxcbjMuICoqQ29udGVudDoqKlxcbiAgIC0gUmVwbGFjZSB0aGUgY29tbWVudCBgPCEtLSBZb3VyIHBvcnRmb2xpbyBjb250ZW50IGhlcmUgLS0+YCB3aXRoIHlvdXIgYWN0dWFsIHBvcnRmb2xpbyBjb250ZW50LCBzdWNoIGFzIHByb2plY3QgZGVzY3JpcHRpb25zLCBza2lsbHMsIG9yIGNvbnRhY3QgaW5mb3JtYXRpb24uXFxuXFxuNC4gKipQbGFjZWhvbGRlcnM6KipcXG4gICAtIFJlbWVtYmVyIHRvIHJlcGxhY2UgdGhlIHBsYWNlaG9sZGVyIGFzdHJvbmF1dCBpbWFnZSBVUkwgd2l0aCB5b3VyIGFjdHVhbCBpbWFnZSBVUkwuXFxuXFxuKipUbyB1c2UgdGhpcyBjb2RlOioqXFxuXFxuMS4gKipDcmVhdGUgYSBuZXcgSFRNTCBmaWxlOioqIFNhdmUgdGhlIGNvZGUgYWJvdmUgYXMgYW4gSFRNTCBmaWxlIChlLmcuLCBgcG9ydGZvbGlvLmh0bWxgKS5cXG4yLiAqKlJlcGxhY2UgdGhlIHBsYWNlaG9sZGVyIGltYWdlIFVSTDoqKiBGaW5kIHRoZSBsaW5lIHdpdGggYGJhY2tncm91bmQtaW1hZ2U6IHVybCgpYCBhbmQgcmVwbGFjZSB0aGUgcGxhY2Vob2xkZXIgVVJMIHdpdGggdGhlIFVSTCBvZiB5b3VyIGFzdHJvbmF1dCBpbWFnZS5cXG4zLiAqKkFkZCB5b3VyIGNvbnRlbnQ6KiogUmVwbGFjZSB0aGUgYDwhLS0gWW91ciBwb3J0Zm9saW8gY29udGVudCBoZXJlIC0tPmAgY29tbWVudCB3aXRoIHlvdXIgYWN0dWFsIHBvcnRmb2xpbyBpbmZvcm1hdGlvbi5cXG40LiAqKk9wZW4gdGhlIEhUTUwgZmlsZSBpbiBhIHdlYiBicm93c2VyOioqIFlvdSBjYW4gbm93IHZpZXcgeW91ciBwb3J0Zm9saW8gd2Vic2l0ZSBpbiB0aGUgYnJvd3Nlci5cXG5cXG5UaGlzIHdpbGwgZ2l2ZSB5b3UgYSBzaW1wbGUsIGJsYWNrIGFuZCBibHVlIHBvcnRmb2xpbyB3ZWJzaXRlIHdpdGggYW4gYXN0cm9uYXV0IGJhY2tncm91bmQuIFJlbWVtYmVyIHRvIGN1c3RvbWl6ZSBpdCBmdXJ0aGVyIHRvIHJlZmxlY3QgeW91ciBvd24gc3R5bGUgYW5kIGNvbnRlbnQhXFxuIn1dfV0=' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo2NCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

In [4]:
gais_contents

[{'role': 'user',
  'parts': [{'text': 'Create a web app called Portfolio\\n1.navigation bar containing project , contact me , services.\\n2. It should be black in blue in color , background should contain an astronout picture .\\n\\n4. It should be a single HTML file, with no separate JS or CSS files.\\n5. It should say rights reserved in the footer.\\n6. Do not use any unicode characters.'}]},
 {'role': 'model',
  'parts': [{'text': '```html\\n<!DOCTYPE html>\\n<html>\\n<head>\\n<title>Portfolio</title>\\n<style>\\nbody {\\n  background-image: url(\\"https://images.unsplash.com/photo-1636571091212-1c0647e04115?ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxzZWFyY2h8Mnx8YXN0cm9uYXUgfHxlbnwwfHwwfHw%3D&auto=format&fit=crop&w=500&q=60\\"); /* Replace with your astronaut image URL */\\n  background-size: cover;\\n  background-repeat: no-repeat;\\n  color: white;\\n  font-family: sans-serif;\\n  margin: 0;\\n}\\n\\n.container {\\n  max-width: 800px;\\n  margin: 0 auto;\\n  padding: 20px;\\n}\\n\\nhead

In [5]:
generation_config

{'temperature': 1, 'top_p': 0.95, 'top_k': 64, 'max_output_tokens': 8192}

In [6]:
safety_settings

{}

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [7]:
# @title `show_file` function
drive = None
def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")

In [8]:
for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

Role: user 

Create a web app called Portfolio\n1.navigation bar containing project , contact me , services.\n2. It should be black in blue in color , background should contain an astronout picture .\n\n4. It should be a single HTML file, with no separate JS or CSS files.\n5. It should say rights reserved in the footer.\n6. Do not use any unicode characters. 

-------------------------------------------------------------------------------- 

Role: model 

```html\n<!DOCTYPE html>\n<html>\n<head>\n<title>Portfolio</title>\n<style>\nbody {\n  background-image: url(\"https://images.unsplash.com/photo-1636571091212-1c0647e04115?ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxzZWFyY2h8Mnx8YXN0cm9uYXUgfHxlbnwwfHwwfHw%3D&auto=format&fit=crop&w=500&q=60\"); /* Replace with your astronaut image URL */\n  background-size: cover;\n  background-repeat: no-repeat;\n  color: white;\n  font-family: sans-serif;\n  margin: 0;\n}\n\n.container {\n  max-width: 800px;\n  margin: 0 auto;\n  padding: 20px;\n}\n\nheader 

## Convert & upload files

For each file, Google AI Studio either sent:
- a Google Drive IDs
- a URL, or
- the raw bytes (`inline_data`).

The API itself onlty understands two ways of sending files:

- `inline_data` - where the bytes are placed inline in the request.
- `file_data` - where the file is uploaded to the Files API, and you pass a reference to that file.

This section goes through the `contents` from Google AI Studio, and uploads the file data, to the Files API, so Gemini can access it:

In [ ]:
# @title `upload_file` function

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)

drive = None
def upload_file_data(file_data):
    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        hash = hashlib.sha256(data).hexdigest()
        path = tempfiles / hash
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files"in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

In [ ]:
contents = copy.deepcopy(gais_contents)

for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data)

Here is the coverted `Content`s:

In [ ]:
contents

## Call `generate_content`

In [ ]:
# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=False,
)

In [ ]:
if generation_config.get("candidate_count", 1) == 1:
    display(Markdown(response.text))

In [ ]:
response.candidates

## Or Create a chat

In [ ]:
gemini = genai.GenerativeModel(
    model_name=model,
    generation_config=generation_config,
    safety_settings=safety_settings,
)

A `ChatSession`, should always end with the model's turn.

So before creating the `chat` check whos turn was last.

If the user was last, attach all but the last content as the `history` and send the last content with `send_message` to get the model's response.

If the model was last, put the whole contents list in the history.

In [ ]:
model_turn = contents[-1].get("role", None) == "user"

if model_turn:
    chat = gemini.start_chat(history=contents[:-1])

    response = chat.send_message(contents[-1])

    if generation_config.get("candidate_count", 1) == 1:
        display(Markdown(response.text))
else:
    chat = gemini.start_chat(history=contents)

In [ ]:
if model_turn:
    response.candidates

After that use `send_message` to continue the conversation.

In [ ]:
# response = chat.send_message("Interesting, tell me more.")
# display(Markdown(response.text))